#### Exercise 1.1

##### Read in the meteorite data from the `Meteorite_Landings.csv` file, rename the `mass (g)` column to `mass`, and drop all the latitude and longitude columns. Sort the result by mass in descending order.

In [102]:
import pandas as pd

df = pd.read_csv('data/Meteorite_Landings.csv')
df = df.rename(columns = {'mass (g)': 'mass'})
df = df.drop(columns=['reclat', 'reclong'])

In [69]:
assert type(df) == pd.core.frame.DataFrame
assert 'mass' in df.columns
assert 'reclong' not in df.columns
assert 'reclat' not in df.columns

#### Exercise-1.2

##### Using the meteorite data from the `Meteorite_Landings.csv` file, update the `year` column to only contain the year, convert it to a numeric data type, and create a new column indicating whether the meteorite was observed falling before 1970. Set the index to the `id` column and extract all the rows with IDs between 10,036 and 10,040 (inclusive) with `loc[]`.

###### **Hint 1**: Use `year.str.slice()` to grab a substring.

###### **Hint 2**: Make sure to sort the index before using `loc[]` to select the range.

###### **Bonus**: There's a data entry error in the `year` column. Can you find it? (Don't spend too much time on this.)

In [103]:
df = df.assign(
    year = lambda x: pd.to_numeric(x.year.str.slice(6,10)),
    before1970 = lambda x: x.year < 1970
)
df.set_index('id', inplace=True)
df = df.sort_index()
subset = df.loc[10036:10040]

In [104]:
assert df.year.dtype == 'float64'
assert df.index.name == 'id'
subset = subset.rename(columns = {'mass (g)': 'mass'})
assert subset.mass.mean() == 33964.25

#### Exercise-1.3

##### Using the meteorite data from the `Meteorite_Landings.csv` file, create a pivot table that shows both the number of meteorites and the 95th percentile of meteorite mass for those that were found versus observed falling per year from 2005 through 2009 (inclusive). 

Hint 1: Be sure to convert the `year` column to a number as we did in the previous exercise.

Hint 2: There are missing values on mass.

In [168]:
import numpy as np

df = pd.read_csv('data/Meteorite_Landings.csv')
df = df.rename(columns = {'mass (g)': 'mass'})
df = df.assign(
    year = lambda x: pd.to_numeric(x.year.str.slice(6,10)),
    before1970 = lambda x: x.year < 1970,
    mass = lambda x: x.mass.fillna(0)
)
#df.mass = df.mass.fillna(0)

df_pivoted = df[(df.year >= 2005) & (df.year <= 2009)].pivot_table(
    index='year', values='mass', aggfunc=['count', lambda x:np.percentile(x,95)]
)

In [177]:
df_pivoted.loc[2006][1]

1722.5500000000002

In [178]:
assert df_pivoted.loc[2006][0] == 2456
assert int(df_pivoted.loc[2006][1]) == 1722